Fake News Classifier Using LSTM
data

In [82]:
# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [83]:
df=pd.read_csv('/content/WELFake_Dataset.csv')

In [84]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [85]:
df.shape

(72134, 4)

In [86]:
# check null values
df.isnull().sum()


,0
Unnamed: 0,0
title,558
text,39
label,0


In [87]:
# drop null values
df.dropna(inplace=True)
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [88]:
# check vaklue counts
df['label'].value_counts()

# this is a balance dataset as equal number of 0 and 1

,count
label,
1,36509
0,35028


In [89]:
#drop unnamed column
df.drop(columns=['Unnamed: 0'],inplace=True)
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [90]:
# droping title row where length is more than 20
# it will help us to padding later on
df = df[df['title'].str.split().str.len() <= 20]


In [91]:
df.shape

(68785, 3)

In [92]:
# distribute independent and dependent features
X=df.drop('label',axis=1)
y=df['label']

In [93]:
# shape of X and y
X.shape,y.shape

((68785, 2), (68785,))

In [94]:
# import tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [95]:
## view the vocabulary size
voc_size=5000

#### One Hot Representation


In [96]:
messages=X.copy()

In [97]:
messages

,title,text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n..."
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...


In [98]:
#reset the index
messages.reset_index(inplace=True)

In [99]:
messages['title'][1]

'UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO]'

In [100]:
import nltk
import re
from nltk.corpus import stopwords

In [101]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [102]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []  ## list of all data
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) ## remove all the special characters
    review = review.lower() ## lower casing
    review = review.split() ## spliting the data

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [103]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [104]:
one_hot_repr=[one_hot(words,voc_size)for words in corpus]
one_hot_repr

[[4870, 300, 2461, 2350, 1573, 1590, 319, 4955, 3809, 884, 537, 3799],
 [114,
  2742,
  778,
  4674,
  4986,
  2930,
  4220,
  250,
  510,
  46,
  4017,
  3879,
  4644,
  3799],
 [2479, 2463, 4468, 4321, 4298, 3078, 1958, 3606, 846, 3649, 95, 3302],
 [4566, 4165, 1486, 4733, 3768, 3941, 3202, 3074, 2464, 2587, 851],
 [642, 1958, 3945, 2720, 4091, 459, 2470, 2800, 3945],
 [2161, 93, 1764, 140, 1907, 3045, 4965, 3292, 997, 619, 1892],
 [4280, 592, 2444, 3303, 4165, 377, 3078, 1431, 2972, 3799],
 [633,
  2422,
  4210,
  3993,
  2292,
  2452,
  3984,
  3913,
  2335,
  633,
  1891,
  1290,
  3174,
  3900,
  3799],
 [987, 2051, 295, 3082, 1998, 1639, 3484, 2051],
 [1030, 1180, 3166, 943, 2894, 2844, 2525, 3503],
 [1647, 4337, 2394, 3040, 3953, 2985, 3768, 3118, 3768, 3367],
 [4970, 3050, 3303, 1654, 4979, 3938, 1288, 1309, 1932],
 [491, 1588, 1805, 3050, 4075, 3404, 281, 2361, 2420, 1081, 1780],
 [3115, 2564, 4278, 4944, 4351, 2042, 3090, 4982, 713, 4717],
 [846, 1627, 4078, 524, 2019, 4280,

In [105]:
#compare corpus to its  correnponding one hot representation
corpus[1],one_hot_repr[1]

('unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 [114,
  2742,
  778,
  4674,
  4986,
  2930,
  4220,
  250,
  510,
  46,
  4017,
  3879,
  4644,
  3799])

#### Embedding Representation

In [106]:
#check most long one hot value of one words
sent_length=20
embedded_docs=pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  884  537 3799]
 [   0    0    0 ... 3879 4644 3799]
 [   0    0    0 ... 3649   95 3302]
 ...
 [   0    0    0 ... 4604 2265  713]
 [   0    0    0 ... 2165  564  160]
 [   0    0    0 ... 1640 2361 3762]]


In [107]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0, 4870,  300, 2461,
       2350, 1573, 1590,  319, 4955, 3809,  884,  537, 3799], dtype=int32)

## Creating model


In [108]:
embedding_vector_feature=30
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_shape=(sent_length,)))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 20, 30)         │       150,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100)            │        52,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,501 (791.02 KB)

 Trainable params: 202,501 (791.02 KB)

 Non-trainable params: 0 (0.00 B)

None


In [109]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [110]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


#### Model Training

In [111]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=70)

Epoch 1/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7917 - loss: 0.4157 - val_accuracy: 0.8828 - val_loss: 0.2728
Epoch 2/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9036 - loss: 0.2343 - val_accuracy: 0.8935 - val_loss: 0.2586
Epoch 3/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9217 - loss: 0.1984 - val_accuracy: 0.8916 - val_loss: 0.2607
Epoch 4/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9348 - loss: 0.1721 - val_accuracy: 0.8902 - val_loss: 0.2810
Epoch 5/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9420 - loss: 0.1514 - val_accuracy: 0.8906 - val_loss: 0.2851
Epoch 6/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9511 - loss: 0.1286 - val_accuracy: 0.8909 - val_loss: 0.3105
Epoch 7/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9583 - loss: 0.1123 - val_accuracy: 0.8894 - val_loss: 0.3337
Epoch 8/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9650 - loss: 0.0968 - val_accuracy: 0.

In [122]:
y_pred=model.predict(X_test)

y_pred=np.where(y_pred > 0.5, 1,0)

710/710 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [123]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[ 9966,  1572],
       [  952, 10210]])

In [124]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8888105726872246

In [113]:
# adding dropout layer to prevent overfitting
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_feature=30
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_shape=(sent_length,)))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 20, 30)         │       150,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 30)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100)            │        52,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,501 (791.02 KB)

 Trainable params: 202,501 (791.02 KB)

 Non-trainable params: 0 (0.00 B)

None


In [125]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=70)

Epoch 1/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9586 - loss: 0.1093 - val_accuracy: 0.8910 - val_loss: 0.3347
Epoch 2/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9619 - loss: 0.1011 - val_accuracy: 0.8905 - val_loss: 0.3630
Epoch 3/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9645 - loss: 0.0944 - val_accuracy: 0.8896 - val_loss: 0.3516
Epoch 4/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9648 - loss: 0.0924 - val_accuracy: 0.8897 - val_loss: 0.3835
Epoch 5/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9668 - loss: 0.0862 - val_accuracy: 0.8912 - val_loss: 0.3818
Epoch 6/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9686 - loss: 0.0819 - val_accuracy: 0.8912 - val_loss: 0.4019
Epoch 7/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9721 - loss: 0.0727 - val_accuracy: 0.8874 - val_loss: 0.4275
Epoch 8/10
659/659 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9720 - loss: 0.0740 - val_accuracy: 0

## Evaluation

In [126]:
y_pred=model.predict(X_test)

710/710 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [130]:
y_pred=np.where(y_pred > 0.6, 1,0)

In [131]:

confusion_matrix(y_test,y_pred)

array([[10349,  1189],
       [ 1345,  9817]])

In [133]:

accuracy_score(y_test,y_pred)


0.8883700440528635

In [134]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89     11538
           1       0.89      0.88      0.89     11162

    accuracy                           0.89     22700
   macro avg       0.89      0.89      0.89     22700
weighted avg       0.89      0.89      0.89     22700

